In [110]:
import pandas as pd
import csv
import re

# Function Declarations 
  
  
### TODOs
- Automate the BigQuery Load Job

In [111]:
def bq_load_job():
    pass

def parse_text_file(csv_filename, file_path):
    """
    Creates a CSV file and parses a text file line-by-line and inserts in comma separated format.
    This function assumes that values are separated by spaces.

    Args:
    - csv_filename: path to csv file (string)
    - file_path: text file to parse (string)
    """
    csv_file = open(csv_filename, 'w')
    csv_writer = csv.writer(csv_file, delimiter=",")

    with open(file_path) as txt_file:
        for line in txt_file:
            csv_writer.writerow(line.split())
        csv_file.close()


def dms_to_dd(dms):
    """
    Converts DMS (Degrees Minutes Seconds) coordinates separated by "." and converts it into DD (decimal Degress) format.

    Args:
    - DMS formatted coordinate (string)
    """
    # create list by splitting at "." 
    parts = re.split('[\."]+', dms) 
    degrees = parts[0]
    minutes = parts[1]
    seconds = parts[2]
    dd = float(degrees) + float(minutes)/60 + float(seconds)/36000
    return dd


def dd_to_dms(deg):
    """
    Converts DD (decimal Degress) into DMS (Degree Minutes Seconds) coordinates format.

    Args:
    - DD formatted coordinate (float)
    """
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]


def dms_to_dd_dir(dms):
    """
    Converts DMS (Degree Minutes Seconds) coordinates separated by "." and converts it into DD (decimal Degress) format.
    Supports direction.

    Args:
    - DMS formatted coordinate (string)
    """
    parts = re.split('[^\d\w]+', dms)
    degrees = parts[0]
    minutes = parts[1]
    seconds = parts[2]
    direction = parts[3]
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60)
    if direction == 'S' or direction == 'W':
        dd *= -1
    return dd


def dd_to_dms_dir(deg):
    """
    Converts DD (decimal Degress) into DMS (Degree Minutes Seconds) coordinates format. 
    Supports direction.

    Args:
    - DD formatted coordinate (float)
    """
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

# Read Files

In [112]:
parse_text_file('./output/fishdata_dms.csv', './input/unformatted_data.txt')

In [113]:
df = pd.read_csv('./output/fishdata_dms.csv', delimiter=",")
df_dir = pd.read_csv('./input/mixed_locs_dir.csv', delimiter=",")
df_ts = pd.read_csv('./input/teamsvinet.csv', delimiter=",")

# Data Preparation

In [114]:
df['Fish_type'] = df['Fish_type'].str.replace('_', ' ')
df['Location_name'] = df['Location_name'].str.replace('_', ' ')
df = df.append(df_ts)

In [115]:
df['latitude'] = df['latitude'].apply(lambda x: dms_to_dd(x))
df['longitude'] = df['longitude'].apply(lambda x: dms_to_dd(x))
df

,Fish_type,Location_name,latitude,longitude
0,Bornholm Laks,31’eren,55.408333,14.986111
1,Bornholm Laks,Anders Sten,55.572222,15.150000
2,Bornholm Laks,Davids Banke,55.366667,14.683333
3,Bornholm Laks,Don’s Knold,55.444444,14.902778
4,Bornholm Laks,Gravstenen øst,55.483333,14.925000
...,...,...,...,...
22,Torsk 20kg 2007,Allangstaran,70.589472,20.545111
23,Helleflynder 2007,Allangstaran,70.589278,20.562639
24,Helleflynder 2007,Allangstaran,70.597278,20.560611
25,Helleflynder 2007,Allangstaran,70.602889,20.546944


In [116]:
df_dir['Fish_type'] = df_dir['Fish_type'].str.replace('_', ' ')
df_dir['Location_name'] = df_dir['Location_name'].str.replace('_', ' ')
df_dir

,Fish_type,Location_name,latitude,longitude
0,unknown,S/S Hugo Stinnes,56°22’203 N,008°04’353 E
1,unknown,Lille vrag NV Hugo Stinnes,56°22’779 N,008°03’808 E
2,unknown,Stenpolle med rester af et flyvrag,56°19’319 N,007°57’026 E
3,unknown,Stentop,56°21’690 N,007°57’650 E
4,unknown,Stendys,56°22’200 N,007°54’079 E
5,unknown,Ballaststen fra sejlskib,56°23’150 N,007°45’554 E
6,unknown,Hummer sten,56°27’362 N,007°51’253 E
7,unknown,Brostensvrag norsk Glitne,56°27’860 N,007°59’630 E
8,unknown,Jernpram med spil,56°28’110 N,007°22’440 E
9,unknown,Verdens største flydekran da den sank lige eft...,56°27’730 N,007°02’640 E


In [117]:
df_dir['latitude'] = df_dir['latitude'].apply(lambda x: dms_to_dd_dir(x))
df_dir['longitude'] = df_dir['longitude'].apply(lambda x: dms_to_dd_dir(x))
df_dir

,Fish_type,Location_name,latitude,longitude
0,unknown,S/S Hugo Stinnes,56.423056,8.164722
1,unknown,Lille vrag NV Hugo Stinnes,56.583056,8.274444
2,unknown,Stenpolle med rester af et flyvrag,56.405278,7.957222
3,unknown,Stentop,56.541667,8.130556
4,unknown,Stendys,56.422222,7.921944
5,unknown,Ballaststen fra sejlskib,56.425000,7.903889
6,unknown,Hummer sten,56.550556,7.920278
7,unknown,Brostensvrag norsk Glitne,56.688889,8.158333
8,unknown,Jernpram med spil,56.497222,7.488889
9,unknown,Verdens største flydekran da den sank lige eft...,56.652778,7.211111


In [118]:
df = df.append(df_dir, ignore_index=True)
df.to_csv('./output/fishdata_dd.csv', index=False)